<a href="https://colab.research.google.com/github/neeluvermaiitj/Dependable-AI/blob/main/E_Accuracy_after_bias_detection_and_mitigation_on_cipher10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
  # Complete dataset 
  import keras
  from keras import layers
  from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPool2D, Flatten, BatchNormalization,Input,MaxPooling2D,Dense,Flatten,Dropout
  from keras.layers import Conv1D, MaxPool1D, Reshape
  from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate
  from keras.models import Model, Sequential,load_model
  from keras.datasets import cifar10
  import tensorflow as tf
  from keras.optimizers import SGD, Adam, RMSprop, Adadelta
  from keras.utils import to_categorical
  import scipy.io as sio
  from scipy.io import loadmat
  import random
  from keras.objectives import mean_squared_error
  import keras.backend as K
  import numpy as np
  import matplotlib.pyplot as plt
  from sklearn import datasets, svm, metrics
  from keras.datasets import cifar10

  def shuffle_in_unison(a, b):
      assert len(a) == len(b)
      shuffled_a = np.empty(a.shape, dtype=a.dtype)
      shuffled_b = np.empty(b.shape, dtype=b.dtype)
      permutation = np.random.permutation(len(a))
      for old_index, new_index in enumerate(permutation):
          shuffled_a[new_index] = a[old_index]
          shuffled_b[new_index] = b[old_index]
      return shuffled_a, shuffled_b
  # scale pixels    
  def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm 
  def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY,testX, testY

  # define cnn model
  def define_model():
    model = Sequential()
    model.add(Conv2D(input_shape=(32, 32, 3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
  
  def create_dataset():  
    
    if load_dataset_saved == 0:
        # load dataset
        trainX, trainY, testX, testY = load_dataset()
        # prepare pixel data
        trainX, testX = prep_pixels(trainX, testX)

        MAT_save_name = 'trainXlinf_cnn_pert.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()  
        trainXlinf = annots['mydata']
        
        MAT_save_name = 'fgsm_cnn_pert_train.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()
        trainXFGSM = annots['mydata']

        trainX = np.concatenate([np.array(trainX), np.array(trainXFGSM), np.array(trainXlinf)])
        #trainX = np.concatenate([np.array(trainX), np.array(trainXlinf)])
        trainY = np.concatenate([np.array(trainY), np.array(trainY), np.array(trainY)])

        MAT_save_name = 'fgsm_cnn_pert.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()  
        testXadv = annots['mydata']
        
        MAT_save_name = 'testXlinf_cnn_pert.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()  
        testXlinf = annots['mydata']

        testX = np.concatenate([np.array(testX), np.array(testXadv), np.array(testXlinf)])
        testY = np.concatenate([np.array(testY), np.array(testY), np.array(testY)])

        MAT_save_name = 'RetrainX.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        sio.savemat(path, {'mydata': trainX})

        MAT_save_name = 'RetrainY.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        sio.savemat(path, {'mydata': trainY})
        
        MAT_save_name = 'RetestX.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        sio.savemat(path, {'mydata': testX})

        MAT_save_name = 'RetestY.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        sio.savemat(path, {'mydata': testY})

        print("saved dataset")

    else:
        MAT_save_name = 'RetrainX.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()
        trainX = annots['mydata']

        MAT_save_name = 'RetrainY.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()
        trainY = annots['mydata']

        MAT_save_name = 'RetestX.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()
        testX = annots['mydata']

        MAT_save_name = 'RetestY.mat'
        path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
        annots = loadmat(path)
        annots.keys()
        testY = annots['mydata']
    trainX, trainY = shuffle_in_unison(trainX, trainY)
    testX, testY = shuffle_in_unison(testX, testY)
    return trainX, trainY, testX, testY

  # run the test harness for evaluating a model
  def run_test_harness():
    trainX, trainY, testX, testY = create_dataset()
    print(trainX.shape, trainY.shape, testX.shape, testY.shape)
    if load_trained_model == 0:
      # define model
      model = define_model()
      print(model)
      # fit model
      history = model.fit(trainX, trainY, epochs=10, batch_size=64, validation_data=(testX, testY), verbose=1)
      model_save_name = 'retrained_model.h5'
      path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{model_save_name}" 
      model.save(path)
    else:

      model_save_name = 'retrained_model.h5'
      path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{model_save_name}" 
      model = load_model(path)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=1)
    print('> %.3f' % (acc * 100.0))
  # entry point, run the test harness
  labels = [  'airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
  tf.compat.v1.disable_eager_execution()
  loss_object = tf.keras.losses.CategoricalCrossentropy()
  load_dataset_saved  = 1
  load_trained_model = 0
  run_test_harness()

(150000, 32, 32, 3) (150000, 10) (30000, 32, 32, 3) (30000, 10)
Train on 150000 samples, validate on 30000 samples
Epoch 1/10
150000/150000 [==============================] - ETA: 0s - loss: 2.0977 - accuracy: 0.2034

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


150000/150000 [==============================] - 124s 825us/sample - loss: 2.0977 - accuracy: 0.2034 - val_loss: 1.6843 - val_accuracy: 0.3453
Epoch 2/10
150000/150000 [==============================] - 115s 769us/sample - loss: 1.4345 - accuracy: 0.4682 - val_loss: 1.2101 - val_accuracy: 0.5619
Epoch 3/10
150000/150000 [==============================] - 116s 775us/sample - loss: 1.0914 - accuracy: 0.6067 - val_loss: 1.0258 - val_accuracy: 0.6372
Epoch 4/10
150000/150000 [==============================] - 114s 761us/sample - loss: 0.8484 - accuracy: 0.6987 - val_loss: 0.8948 - val_accuracy: 0.6871
Epoch 5/10
150000/150000 [==============================] - 114s 758us/sample - loss: 0.6535 - accuracy: 0.7706 - val_loss: 0.9025 - val_accuracy: 0.6899
Epoch 6/10
150000/150000 [==============================] - 115s 767us/sample - loss: 0.4832 - accuracy: 0.8309 - val_loss: 0.9960 - val_accuracy: 0.6955
Epoch 7/10
150000/150000 [==============================] - 115s 767us/sample - loss: 0

In [ ]:


  # Complete dataset 
  import keras
  from keras import layers
  from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPool2D, Flatten, BatchNormalization,Input,MaxPooling2D,Dense,Flatten,Dropout
  from keras.layers import Conv1D, MaxPool1D, Reshape
  from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate
  from keras.models import Model, Sequential,load_model
  from keras.datasets import cifar10
  import tensorflow as tf
  from keras.optimizers import SGD, Adam, RMSprop, Adadelta
  from keras.utils import to_categorical
  import scipy.io as sio
  from scipy.io import loadmat
  import random
  from keras.objectives import mean_squared_error
  import keras.backend as K
  import numpy as np
  import matplotlib.pyplot as plt
  from sklearn import datasets, svm, metrics
  from keras.datasets import cifar10

  # scale pixels    
  def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm 
  def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY,testX, testY

  # define cnn model
  def define_model():
    model = Sequential()
    model.add(Conv2D(input_shape=(32, 32, 3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
  
  # run the test harness for evaluating a model
  def run_test_harness():
      # load dataset
      trainX, trainY, testX, testY = load_dataset()
      # prepare pixel data
      trainX, testX = prep_pixels(trainX, testX)
      
      model_save_name = 'retrained_model.h5'
      path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{model_save_name}" 
      model = load_model(path)

      # evaluate model
      _, acc = model.evaluate(testX, testY, verbose=1)
      print(' Performance on Oriinal Dataset > %.3f' % (acc * 100.0))

      MAT_save_name = 'fgsm_cnn_pert.mat'
      path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
      annots = loadmat(path)
      annots.keys()
      testX = annots['mydata']

      # evaluate model
      _, acc = model.evaluate(testX, testY, verbose=1)
      print(' Performance on FGSM Dataset > %.3f' % (acc * 100.0))

      MAT_save_name = 'testXlinf_cnn_pert.mat'
      path = F"/content/drive/MyDrive/Colab Notebooks/DAI_Assignment2/cifar10_code/{MAT_save_name}"
      annots = loadmat(path)
      annots.keys()
      testX = annots['mydata']

      # evaluate model
      _, acc = model.evaluate(testX, testY, verbose=1)
      print(' Performance on L-Inf Dataset > %.3f' % (acc * 100.0))

  run_test_harness()

313/313 [==============================] - 105s 335ms/step - loss: 1.0293 - accuracy: 0.7390
 Performance on Oriinal Dataset > 73.900
313/313 [==============================] - 105s 336ms/step - loss: 1.2213 - accuracy: 0.6908
 Performance on FGSM Dataset > 69.080
313/313 [==============================] - 105s 334ms/step - loss: 1.0373 - accuracy: 0.7380
 Performance on L-Inf Dataset > 73.800
